# TRIDENT – end-to-end example (pairs and triplets)

This notebook demonstrates the full TRIDENT pipeline:

1. Load a binary bipartite matrix $A$ (e.g., regions × time points).
2. Fit **BiCM** and validate **pairs** (two-star overlaps).
3. Fit **BiPLOM** and validate **triplets** (three-star overlaps).

The workflow is domain-agnostic: any binary two-mode dataset can be analysed in the same way.


In [2]:
import numpy as np
import pandas as pd

from scipy.special import comb

from pathlib import Path
import sys

# Add <repo_root>/src to PYTHONPATH so that `import trident` works
repo_root = Path.cwd().resolve().parent          # notebooks/ -> repo root
src_path = repo_root / "src"

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import trident
print("TRIDENT imported from:", trident.__file__)

from trident import (
    bicm_solver,
    probability_matrix_from_bicm,
    biplom_solver,
    probability_matrix_from_biplom,
    TwoStarValidator,
    TripletValidator,
)

TRIDENT imported from: /Users/mattia/Desktop/TRIDENT/src/trident/__init__.py


## 1) Load a binary bipartite matrix

`A` must be an `N×M` binary array (integers 0/1). Rows correspond to one layer (e.g., ROIs), columns to the other (e.g., time points).


In [3]:
# Path relative to this notebook
path = "./test_dataset.txt"

A = pd.read_csv(path, header=None, sep=r"\s+", dtype=int).values
A = np.asarray(A, dtype=np.int8)

N, M = A.shape
print("Loaded bipartite matrix A with shape:", (N, M))
print("Density (ones / total):", A.mean())

# Optional: basic sanity checks
assert set(np.unique(A)).issubset({0, 1}), "A must be binary (0/1)."
assert (A.sum(axis=1) > 0).all(), "Some rows are all-zero; BiCM/BiPLOM may become ill-conditioned."
assert (A.sum(axis=0) > 0).all(), "Some columns are all-zero; BiCM/BiPLOM may become ill-conditioned."

row_names = [f"row_{i}" for i in range(N)]

def _rel_err(a, b, eps=1e-12):
    """Return elementwise relative error |a-b| / max(eps, |b|)."""
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    return np.abs(a - b) / np.maximum(eps, np.abs(b))

Loaded bipartite matrix A with shape: (116, 2396)
Density (ones / total): 0.2969172759196362


## 2) Fit BiCM and validate pairs

We first estimate the BiCM parameters and build the corresponding probability matrix $P^{\mathrm{BiCM}}$.
Then we validate pairwise overlaps $V_{ij}=\sum_{t} b_{it} b_{jt}$ via a Poisson–Binomial null distribution, using a **Gaussian approximation** (`method="normal"`).


In [4]:
# Fit BiCM
bicm_res = bicm_solver(A)
P_bicm = probability_matrix_from_bicm(bicm_res)

print("BiCM probability matrix shape:", P_bicm.shape)
print("P range:", float(P_bicm.min()), float(P_bicm.max()))

k_obs = A.sum(axis=1)          # row degrees
h_obs = A.sum(axis=0)          # column degrees
k_exp = P_bicm.sum(axis=1)
h_exp = P_bicm.sum(axis=0)

err_k = _rel_err(k_exp, k_obs)
err_h = _rel_err(h_exp, h_obs)

print(
    "BiCM constraints | "
    f"rows max/mean rel.err = {err_k.max():.2e}/{err_k.mean():.2e}, "
    f"cols max/mean rel.err = {err_h.max():.2e}/{err_h.mean():.2e}"
)

BiCM probability matrix shape: (116, 2396)
P range: 0.013808279287133959 0.9852817107768788
BiCM constraints | rows max/mean rel.err = 9.73e-11/4.12e-11, cols max/mean rel.err = 9.74e-11/3.67e-11


In [ ]:
# Validate pairs (two-star / V-motifs)
val_pairs = TwoStarValidator(
    A,
    P_bicm,
    n_jobs=-1,          # parallel execution (set 1 for serial)
    progress=True,
)

pvals_pairs = val_pairs.compute_pvals(method="normal")

df_pairs, thr_pairs = val_pairs.validate(
    pvals_pairs,
    alpha=0.05,
    method="fdr",
    names=row_names,
)

print("All possible pairs:", int(comb(N, 2)))
print("Validated pairs:", len(df_pairs), f"(FDR threshold p ≤ {thr_pairs:.3g})")

df_pairs.head()

  0%|          | 0/6670 [00:00<?, ?it/s]

All possible pairs: 6670
Validated pairs: 1888 (FDR threshold p ≤ 0.0141)


,i,j,p
0,row_3,row_11,0.000232
1,row_3,row_27,0.000277
2,row_3,row_30,0.000394
3,row_3,row_43,0.003546
4,row_3,row_92,0.003653


## 3) Fit BiPLOM and validate triplets

We now fit the second-order null model **BiPLOM**, build $P^{\mathrm{BiPLOM}}$, and validate triplet overlaps
$V_{ijk}=\sum_{t} b_{it} b_{jt} b_{kt}$.

Again, we use a **Gaussian approximation** (`method="normal"`) and apply FDR correction.


In [6]:
# Fit BiPLOM
biplom_res = biplom_solver(A)
P_biplom = probability_matrix_from_biplom(biplom_res)

print("BiPLOM probability matrix shape:", P_biplom.shape)
print("P range:", float(P_biplom.min()), float(P_biplom.max()))

k_exp2 = P_biplom.sum(axis=1)
h_exp2 = P_biplom.sum(axis=0)

# ------------------------------------------------------------
# BiPLOM additional constraint in the "tilde" approximation:
#   V~_i = sum_{j,t} p_it p_jt = sum_t p_it (sum_j p_jt)
# This includes the j=i contribution and avoids the degeneracy
# discussed in the paper when enforcing j != i at finite size.
# ------------------------------------------------------------
colsum = P_biplom.sum(axis=0)          # s_t = sum_j p_jt
Vtilde_exp = (P_biplom * colsum).sum(axis=1)

# Empirical counterpart:
#   V~*_i = sum_{j,t} a_it a_jt = sum_t a_it (sum_j a_jt)
colsum_A = A.sum(axis=0)              # h_t = sum_j a_jt
Vtilde_obs = (A * colsum_A).sum(axis=1)

err_k2 = _rel_err(k_exp2, k_obs)
err_h2 = _rel_err(h_exp2, h_obs)
err_Vt = _rel_err(Vtilde_exp, Vtilde_obs)

print(
    "BiPLOM constraints (tilde) | "
    f"rows max/mean rel.err = {err_k2.max():.2e}/{err_k2.mean():.2e}, "
    f"cols max/mean rel.err = {err_h2.max():.2e}/{err_h2.mean():.2e}, "
    f"Vtilde max/mean rel.err = {err_Vt.max():.2e}/{err_Vt.mean():.2e}"
)

Using Hessian regularisation in exp mode for numerical stability.
Setting initial guess
max rows error = 7.960579750942998e-09
max columns error = 5.8317937146057375e-08
max V error = 6.94293703418225e-07
Solver converged.
BiPLOM probability matrix shape: (116, 2396)
P range: 0.0015222136822555582 0.9998194871937797
BiPLOM constraints (tilde) | rows max/mean rel.err = 1.27e-11/1.63e-12, cols max/mean rel.err = 5.16e-10/1.02e-12, Vtilde max/mean rel.err = 2.55e-11/7.57e-12


In [9]:
# Validate triplets (three-star)
val_trip = TripletValidator(
    A,
    P_biplom,
    n_jobs=-1,
    progress=True,
)

pvals_trip = val_trip.compute_pvals(method="normal")

df_trip, thr_trip = val_trip.validate(
    pvals_trip,
    alpha=0.05,
    method="fdr",
    names=row_names,
)

print("All possible triplets:", int(comb(N, 3)))
print("Validated triplets:", len(df_trip), f"(FDR threshold p ≤ {thr_trip:.3g})")

df_trip.head()

  0%|          | 0/253460 [00:00<?, ?it/s]

All possible triplets: 253460
Validated triplets: 23260 (FDR threshold p ≤ 0.00459)


,i,j,k,p
0,row_0,row_1,row_9,0.003524
1,row_0,row_1,row_18,0.003254
2,row_0,row_1,row_28,0.000325
3,row_0,row_1,row_66,0.000006
4,row_0,row_1,row_111,0.000223


## Notes

- For improved tail accuracy you can use `method="rna"` (refined normal approximation) or `method="poibin"` (FFT-based Poisson–Binomial).
- If your matrix contains all-zero rows/columns, remove them before fitting to avoid degeneracies.
